In [ ]:
# UDP Broadcast Client using asyncio (non-blocking)
# Run this in a separate Jupyter notebook

import socket
import json
import datetime
import uuid
import asyncio
import sys

# Create a unique ID for this client
client_id = str(uuid.uuid4())[:8]

# Store received messages for demonstration purposes
received_messages = []
last_processed_id = -1

# Print initialization message
print(f"Client {client_id} starting up...")

# Define the port clients will listen on
client_port = 37020

class UDPClient:
    def __init__(self):
        self.transport = None
        self.response_transport = None
        self.indicator_task = None
        
    def connection_made(self, transport):
        self.transport = transport
        print(f"Client {client_id} started, listening on port {client_port}")
        print("Press Ctrl+C to stop")
        
    def datagram_received(self, data, addr):
        global last_processed_id
        
        try:
            server_ip, server_port = addr
            
            # Decode and parse the message
            message_str = data.decode('utf-8')
            message = json.loads(message_str)
            
            # Extract timestamp and state
            timestamp = message.get("timestamp", "unknown")
            state = message.get("state", "unknown")
            message_id = message.get("message_id", -1)
            
            # Only process new messages (in case of duplicates)
            if message_id > last_processed_id:
                last_processed_id = message_id
                
                # Get current time
                receive_time = datetime.datetime.now().isoformat()
                
                # Print received message (clear the waiting indicator first)
                print(f"\nReceived broadcast from {server_ip}:{server_port}")
                print(f"Message ID: {message_id}")
                print(f"Timestamp: {timestamp}")
                print(f"State: {state}")
                
                # Append to message history
                received_messages.append({
                    "server_ip": server_ip,
                    "server_port": server_port,
                    "timestamp": timestamp,
                    "receive_time": receive_time,
                    "state": state,
                    "message_id": message_id
                })
                
                # Create response socket if needed
                if not self.response_transport:
                    # We'll create it when needed
                    pass
                
                # Send response back to the server
                response = f"Client {client_id} received message {message_id}"
                
                # For responses, we use a new transport created on demand
                loop = asyncio.get_event_loop()
                coro = loop.create_datagram_endpoint(
                    lambda: asyncio.DatagramProtocol(),
                    remote_addr=(server_ip, server_port)
                )
                transport, _ = loop.run_until_complete(coro)
                transport.sendto(response.encode('utf-8'))
                transport.close()
                
                # Print statistics
                print(f"Total messages received: {len(received_messages)}")
                
                # Display the last 5 states for demonstration
                if len(received_messages) >= 5:
                    recent_states = [msg["state"] for msg in received_messages[-5:]]
                    print(f"Last 5 states: {recent_states}")
                
        except json.JSONDecodeError:
            print(f"Error: Received invalid JSON data")
        
        except Exception as e:
            print(f"Error: {e}")
    
    def error_received(self, exc):
        print(f"Error received: {exc}")
    
    def connection_lost(self, exc):
        if exc:
            print(f"Connection lost due to error: {exc}")
        else:
            print("Connection closed")

async def waiting_indicator():
    """Display a waiting indicator while no messages are being received"""
    waiting_chars = ['|', '/', '-', '*']
    i = 0
    try:
        while True:
            sys.stdout.write(f"\rWaiting for broadcasts {waiting_chars[i]} ")
            sys.stdout.flush()
            i = (i + 1) % len(waiting_chars)
            await asyncio.sleep(0.5)
    except asyncio.CancelledError:
        # Clear the indicator when cancelled
        sys.stdout.write("\r                          \r")
        sys.stdout.flush()

async def main():
    """Main coroutine that sets up the client and handles cleanup"""
    # Get a reference to the event loop
    loop = asyncio.get_event_loop()
    
    # Create UDP endpoint
    # Set up socket for use with asyncio
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    sock.bind(('0.0.0.0', client_port))
    
    # Create endpoint with pre-configured socket
    transport, protocol = await loop.create_datagram_endpoint(
        lambda: UDPClient(),
        sock=sock
    )
    
    # Start the waiting indicator
    indicator_task = asyncio.create_task(waiting_indicator())
    
    try:
        # This will keep running until interrupted
        await asyncio.sleep(float('inf'))
    except asyncio.CancelledError:
        print("\nShutting down...")
    finally:
        # Clean up
        indicator_task.cancel()
        try:
            await indicator_task
        except asyncio.CancelledError:
            pass
        
        transport.close()
        print("Client stopped")
        
        # Print summary
        print(f"\nSummary: Received {len(received_messages)} messages")

# Run the client
try:
    # This approach works well in Jupyter notebooks 
    # which already have an asyncio event loop running
    
    # Method 1: For regular Python script
    # asyncio.run(main())
    
    # Method 2: Better for Jupyter notebooks
    loop = asyncio.get_event_loop()
    task = loop.create_task(main())
    
    # The following is not necessary in a Jupyter notebook
    # loop.run_until_complete(task)
    
    print("Client is running. In Jupyter, use interrupt kernel to stop.")
    
except KeyboardInterrupt:
    print("\nClient interrupted by user")